# Gated Recurrent Units and Further Topics in NMT

## GRU

![006Fmjmcly1fh26tkuf3mj31ay0r8qat.jpg](https://i.loli.net/2020/03/07/F4xOnUL1wZ9zMCP.jpg)

上图为RNN，使用h_t预测h_{t+1}，再将h_{t+1}反向传播回h_t，但从h_{t+n}到h_t红线连乘多次下溢出。而GRU额外添加了一些“捷径”红线，允许梯度直接流过去，而不是连乘的方式递减过去。

![006Fmjmcly1fh279bre16j318m0ju0zq.jpg](https://i.loli.net/2020/03/07/rFlG9obHTB7w8Zf.jpg)

用来自适应学习应该把多少注意力放到前一个隐藏层状态上。candidate RNN的update形式与普通RNN相似，此外，update gate自适应学习一个0-1的数u_t来决定多少权重在h_t上多少权重在h_{t-1}上，当u_t接近于0时，f接近于h_{t-1}，我们可以认为h_t由h_{t-1}直接反应，这样就允许远处的梯度直接传播而不需要做连续的乘法

![006Fmjmcly1fh2ylvadzrj319u0h6agh.jpg](https://i.loli.net/2020/03/07/1W8ILHJBTCzQESf.jpg)

reset gate通过自适应的学习一个0-1的数r_t来删除一些不必要的h_{t-1}，相当于遗忘掉一部分hidden state

![006Fmjmcly1fh2yo6qq4ej31ca0j6q60.jpg](https://i.loli.net/2020/03/07/lV7BSzfQjRg6dev.jpg)

对于RNN寄存器来说，先读取出所有寄存器h，计算后更新寄存器，没有灵活性

![006Fmjmcly1fh301jbqhij31cq0jg0wn.jpg](https://i.loli.net/2020/03/07/LkfgQxXbP618ic9.jpg)

与之相反，GRU寄存器允许自适应的学习一种灵活性，使用reset gate可以得到一个hidden state的subset，相当于网络有了遗忘的能力，然后就可以进行RNN的计算了，在计算完成后，使用update gate得到一个subset用于更新学到的candidate，总的来说，GRU的两个gate分别选择读和写的两个subset。

直观的来讲，假如说某一类的词在sentence中非常重要，比如一个动词，gate需要记住这个词，记住的方式是hidden state的某几维保持不变，知道出现下一个动词再指定这几维进行更新。比如说在英语中有很多动词短语make up, make out, take up等，当我们看到make这个词时，我们需要将其储存起来，这样我们看到up时就能结合make正确理解其含义

## GRU和LSTM的对比

GRU中的隐藏状态h_t类似于LSTM中的cell-c_t，而LSTM中的隐藏状态h_t其实相当于一个暴露给外部世界的“显状态”。LSTM通过给cell加一个tanh获得非线性的灵活性。另外，观察h_{t−1}之前的系数，这里的r_t其实是“don't reset gate”，ft其实是“don't forget gate”。

![006Fmjmcly1fh370tezfyj30ov0a6wil.jpg](https://i.loli.net/2020/03/07/o7Xsgj9ydU5SRY6.jpg)

## LSTM

宏观上的LSTM Cell：

![006Fmjmcly1fh379ttwk0j30i40iv42d.jpg](https://i.loli.net/2020/03/07/7BbfAl9ueGQdV3g.jpg)

将所有操作都gate起来，方便遗忘甚至忽略一些信息，而不是把所有东西都塞到一起。

![006Fmjmcly1fh37ds6tjlj30hp0ipaeg.jpg](https://i.loli.net/2020/03/07/LZnOvf7EHahuTbX.jpg)

New Memory Cell的计算是一个非线性的过程，与朴素RNN一模一样：

![006Fmjmcly1fh37mpm9ylj30ho0isdju.jpg](https://i.loli.net/2020/03/07/SIOc7uh28UxfD4r.jpg)

最关键之处在于，Memory Cell的更新中有一个加法项直接来自上一刻的Cell，也就是说建立了c_t和c_{t−1}的直接线性连接（与ResNet类似）：

![006Fmjmcly1fh37r3ydazj30hv0in78c.jpg](https://i.loli.net/2020/03/07/BwIQqtWmHpC6L34.jpg)

类似于GRU中的加法，在反向传播的时候允许原封不动地传递残差，也允许不传递残差，总之是自适应的。有了这些改进，LSTM的记忆可以比RNN持续更长的step（大约100）

训练技巧：

- 将递归权值矩阵初始化为正交
- 将其他矩阵初始化为较小的值
- 将forget gate偏置设为1：默认为不遗忘
- 使用自适应的学习率算法：Adam、AdaDelta
- 裁剪梯度的长度为小于1-5
- 在Cell中垂直应用Dropout而不是水平Dropout
- 保持耐心，通常需要训练很长时间

如果想获得额外的2个百分点的效果提升，可以训练多个模型，平均它们的预测。

![006Fmjmcly1fh38u5n7hgj30oi07hmy4.jpg](https://i.loli.net/2020/03/07/lCUSPw2EaJ35cR9.jpg)

## MT evaluation

对于一种语言的句子，交给不同的人会得到不同的翻译结果，对翻译任务来说没有一个标准答案，有些时候还涉及到直译和意译，所以不能根据accuracy来进行评估，所以对于MT任务，主要有以下几种evaluate的方法：

![截屏2020-03-0722.32.16.png](https://i.loli.net/2020/03/07/dsfe7Ek3ULhW58z.png)

后来IBM发明了一种简单有效的评价策略，BLEU

![006Fmjmcly1fh3doomj4kj30im0ta79e.jpg](https://i.loli.net/2020/03/07/LxnuyfvSmCHpOtw.jpg)

通过比较标准译文与机翻译文中NGram的重叠比率（0到1之间）来衡量机翻质量。

为了防止输出大量无意义的the之类来作弊，可以通过Brevity Penalty来防止机翻比译员译文短

一般取4-gram之内参与评测，最终的分值是所有ngram分值的几何平均乘上一个Brevity Penalty：

![006Fmjmcly1fh3dppisxdj30t80oo79p.jpg](https://i.loli.net/2020/03/07/kQqvZ8p4EBbVWtC.jpg)

为了防止某篇机翻实际上很好，可就是跟人类译文用词行文不相似的情况，IBM的论文建议多准备几篇标准答案，这样总会撞上一个。当然了，虽然IBM建议准备4篇，实际测试中经常只有1篇。刚开始的时候，BLEU和人工打分几乎是线性相关（一致）的，而当人们为了BLEU分数费尽心机之后，两者就开始脱钩了。

## 解决大词表问题

大词表问题指的是softmax的计算难度。早期的MT系统会使用较小的词表，但这并不是解决问题，而是逃避问题。另一种思路是，hierarchical softmax，建立树形词表，但这类方法过于复杂，让模型对树形结构敏感而不是对词语本身敏感。另外就是上几次课见过的廉价的NCE，这些方法对GPU都不友好。

![006Fmjmcly1fh3hecak26j30od0etwi2.jpg](https://i.loli.net/2020/03/07/V4MwOagm3vStzDF.jpg)

最新的方法是训练时每次只在词表的一个小子集上训练，因为40%的词语只出现一次，如果把训练数据均分为许多份，则每一份中的稀有词可能都不会在其他语料中出现。如何选择小词表呢？在刚才的方法上更进一步，让用词相似的文章进入同一个子集，这样每个子集的词表就更小了。

测试的时候先雷打不动将前K个最常使用的单词加入备选词表，然后将原文中每个单词可能的前K′个翻译加进去。最后在备选词表上softmax。

![006Fmjmcly1fh3i0897abj31b80sc0wo.jpg](https://i.loli.net/2020/03/07/EDValjTFpyf3S1n.jpg)

其他的一些方法有基于词素去理解例如goooood morning，或者字符级的LSTM（大部分情况下在词语级别做翻译，出现unknown word的时候在词语级别上做beam search）